<a href="https://colab.research.google.com/github/FibGro/Pyspark/blob/main/Structured_Data_RDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

In [2]:
from pyspark import SparkContext, SparkConf


conf = SparkConf().setAppName('KDDCup_Pyspark').setMaster('local[*]')
sc = SparkContext(conf=conf)

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
rdd= sc.textFile('/content/drive/MyDrive/Colab Notebooks/kddcup.data.corrected')

In [13]:
rdd = rdd.repartition(10) # shuffelling

print(sc.defaultParallelism)
rdd.persist()

2


MapPartitionsRDD[33] at coalesce at NativeMethodAccessorImpl.java:0

In [14]:
# Random 10 datas randomly
rdd.takeSample(False, 10, 1234)

['0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,255,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,smurf.',
 '0,udp,other,SF,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,0.00,0.00,0.00,0.00,0.33,0.67,0.00,255,3,0.01,0.68,0.97,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,228,16,1.00,1.00,0.00,0.00,0.07,0.06,0.00,255,16,0.06,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune.',
 '0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,255,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,smurf.',
 '0,icmp,eco_i,SF,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,39,0.00,0.00,0.00,0.00,1.00,0.00,1.00,2,255,1.00,0.00,1.00,0.50,0.00,0.00,0.00,0.00,ipsweep.',
 '0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,144,19,1.00,1.00,0.00,0.00,0.13,0.06,0.00,255,9,0.04,0.07,0.00,0.00,1.00,1.00,0.00,0.00,neptune.',
 '0,icmp,ecr_i,SF,520,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.00,0.00,0.

In [15]:
# Count element

rdd.count()

4898431

In [16]:
# Calculate Ratio of normal connection

normal_rdd = rdd.filter(lambda line : 'normal.'in line)
ratio = normal_rdd.count() /rdd.count()

print("the ratio of normal connection is {} %".format(ratio))

the ratio of normal connection is 0.19859032412623553 %


In [28]:
# Get the list of labels

split_rdd = rdd.map(lambda line: line.split(","))
label_rdd = split_rdd.map(lambda item:item[-1]).distinct()
label_rdd.collect()



['pod.',
 'neptune.',
 'warezclient.',
 'loadmodule.',
 'smurf.',
 'nmap.',
 'spy.',
 'back.',
 'teardrop.',
 'ipsweep.',
 'multihop.',
 'phf.',
 'ftp_write.',
 'guess_passwd.',
 'normal.',
 'land.',
 'satan.',
 'imap.',
 'portsweep.',
 'warezmaster.',
 'rootkit.',
 'perl.',
 'buffer_overflow.']

In [29]:
# Create Key and Values pairs

label_rdd_ = split_rdd.map(lambda x:(x[-1],1))
label_rdd_reduce = label_rdd.reduceByKey(lambda a,b : a+b)

In [30]:
import pandas as pd

Keys = label_rdd_reduce.keys().collect()
Values = label_rdd_reduce.values().collect()

df_label = pd.DataFrame({'Label' : Keys, 'Count': Values})
df_label.sort_values(by = 'Count'  , ascending=False)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 53.0 failed 1 times, most recent failure: Lost task 0.0 in stage 53.0 (TID 214) (8095ac8a18c8 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1237, in process
    out_iter = func(split_index, iterator)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/lib/python3.10/dist-packages/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/lib/python3.10/d

In [36]:
# Get the connection

split_rdd.filter(lambda x: x[13] == '1') \
          .map(lambda x: (x[1], x[4], x[5])) \
          .filter(lambda x :int( x[2] ) > int(x[1]) *500) \
          .take(5)

[('tcp', '296', '507534'),
 ('tcp', '296', '507534'),
 ('tcp', '246', '866032'),
 ('tcp', '433', '1524348'),
 ('tcp', '255', '574784')]

In [39]:
# Get the list protocols that aare normal ad vulnerable to attacks where the is NOT the guess login to destination address


normal_protocol = split_rdd.filter(lambda line: "normal" in line[-1] and line[21] != '1') \
                            .map(lambda line : (line[1], 1)).reduceByKey(lambda a,b : a+b)

attack_protocal = split_rdd.filter(lambda line: "normal" not in line[-1] and line[21] != '1') \
                            .map(lambda line : (line[1], 1)).reduceByKey(lambda a,b : a+b)


normal_keyvalues = pd.DataFrame({'Label' : normal_protocol.keys().collect(), 'Count': normal_protocol.values().collect()})
attack_keyvalues = pd.DataFrame({'Label' : attack_protocal.keys().collect(), 'Count': attack_protocal.values().collect()})


results = normal_keyvalues.merge(attack_keyvalues, on = 'Label')
results.sort_values(by = 'Count'  , ascending=False)





,Label,Count_x,Count_y
2,tcp,764894,1101613
1,udp,191348,2940
0,icmp,12763,2820782


In [53]:
# Get summary statistic for sums of tcp connections to the same destination IP address

# Get summary statistic for sums of tcp connections to the same destination IP address

from pyspark.mllib.stat import Statistics
from pyspark.mllib.linalg import Vectors

summary = Statistics.colStats(split_rdd.filter(lambda line: line[1] == 'tcp') \
                                       .map(lambda line : Vectors.dense(int(line[31])))) # Convert each integer to a dense vector

tcp_mean = summary.mean()

tcp_variance = summary.variance()


In [54]:
print([tcp_mean, tcp_variance])

[array([201.7520146]), array([8231.16262845])]
